In [5]:
# import libs
import keras 
from keras import layers
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

import csv
import matplotlib.pyplot  as plt
%matplotlib inline

In [6]:
#reading images into array
driving_log_dir = './sim_data/20190725/driving_log.csv'

images = []
measurements = []

with open(driving_log_dir) as csvfile:
    reader = csv.reader(csvfile, delimiter=",")
    for line in reader:
        images.append(plt.imread(line[0]))
#         measurement = []
#         for m in line[3:]:
#             measurement.append(m)
#         measurements.append(measurement)
        measurements.append(line[3])
        
images = np.array(images)
measurements = np.array(np.array(measurements))

In [7]:
print(images[0].shape)

(160, 320, 3)


In [11]:
#building baseline model for testing 
def cnn_model(Input):
    x = layers.Lambda(lambda x: (x/255.0)-0.5)(Input)
    x = layers.Cropping2D(cropping=((70,25), (0,0)))(x)
    x = layers.Conv2D(32, (3,3), padding='same', activation='relu')(x)
    x = layers.MaxPool2D((2,2))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, (3,3), padding='same', activation='relu')(x)
    x = layers.MaxPool2D((2,2))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(128, (3,3), padding='same', activation='relu')(x)
    x = layers.MaxPool2D((2,2))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dense(1)(x)
    return x 

In [12]:
img_input = layers.Input(shape=(160,320,3))

In [13]:
output = cnn_model(img_input)

Instructions for updating:
Colocations handled automatically by placer.


In [14]:
model = keras.Model(inputs = img_input, outputs=output)

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 160, 320, 3)       0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 160, 320, 3)       0         
_________________________________________________________________
cropping2d_2 (Cropping2D)    (None, 65, 320, 3)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 65, 320, 32)       896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 160, 32)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 160, 32)       128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 160, 64)       18496     
__________

In [16]:
model.compile(optimizer='adam',
             loss='mse',
             metrics=['mse'])

In [17]:
model.fit(images, measurements,
          validation_split=0.2,
          shuffle=True,
         batch_size=64,
         epochs=10,
         verbose=1)

Instructions for updating:
Use tf.cast instead.
Train on 2383 samples, validate on 596 samples
Epoch 1/10
2383/2383 [==============================] - 10s 4ms/step - loss: 35.6464 - mean_squared_error: 35.6464 - val_loss: 0.0223 - val_mean_squared_error: 0.0223
Epoch 2/10
2383/2383 [==============================] - 6s 2ms/step - loss: 0.1414 - mean_squared_error: 0.1414 - val_loss: 0.0235 - val_mean_squared_error: 0.0235
Epoch 3/10
2383/2383 [==============================] - 6s 2ms/step - loss: 0.0712 - mean_squared_error: 0.0712 - val_loss: 0.0198 - val_mean_squared_error: 0.0198
Epoch 4/10
2383/2383 [==============================] - 6s 3ms/step - loss: 0.0658 - mean_squared_error: 0.0658 - val_loss: 0.0194 - val_mean_squared_error: 0.0194
Epoch 5/10
2383/2383 [==============================] - 7s 3ms/step - loss: 0.0614 - mean_squared_error: 0.0614 - val_loss: 0.0233 - val_mean_squared_error: 0.0233
Epoch 6/10
2383/2383 [==============================] - 7s 3ms/step - loss: 0.0582

In [18]:
model.save('./baseline.h5')